# LangGraph

In [ ]:
!poetry add langgraph

## 准备

### 环境

In [43]:
import os
from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv(), override=True)

True

## 简单的例子

### 定义图

In [1]:
from typing import TypedDict, Annotated, Sequence
import operator
from langchain_core.messages import BaseMessage
from langgraph.graph import StateGraph

class AgentState(TypedDict):
    messages: Annotated[Sequence[BaseMessage], operator.add]

In [2]:
# 定义状态
workflow = StateGraph(AgentState)

### 定义工具集

In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

tools = [TavilySearchResults(max_results=1)]

### 定义 agent 节点

In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0, streaming=True)

In [8]:
from langchain_core.utils.function_calling import convert_to_openai_function

functions = [convert_to_openai_function(t) for t in tools]
model = model.bind(functions=functions)

In [9]:
def call_model(state):
    messages = state['messages']
    response = model.invoke(messages)
    # We return a list, because this will get added to the existing list
    return {"messages": [response]}

In [10]:
# 定义 agent 节点
workflow.add_node("agent", call_model)

### 定义 action 节点

In [11]:
from langchain_core.messages import FunctionMessage
from langgraph.prebuilt import ToolInvocation
import json

from langgraph.prebuilt import ToolExecutor
tool_executor = ToolExecutor(tools)

def call_tool(state):
    messages = state['messages']
    # Based on the continue condition
    # we know the last message involves a function call
    last_message = messages[-1]
    # We construct an ToolInvocation from the function_call
    action = ToolInvocation(
        tool=last_message.additional_kwargs["function_call"]["name"],
        tool_input=json.loads(last_message.additional_kwargs["function_call"]["arguments"]),
    )
    # We call the tool_executor and get back a response
    response = tool_executor.invoke(action)
    # We use the response to create a FunctionMessage
    function_message = FunctionMessage(content=str(response), name=action.tool)
    # We return a list, because this will get added to the existing list
    return {"messages": [function_message]}

In [12]:
# 定义 action 节点
workflow.add_node("action", call_tool)

### 定义条件边

In [13]:
from langgraph.graph import END

def should_continue(state):
    messages = state['messages']
    last_message = messages[-1]
    # If there is no function call, then we finish
    if "function_call" not in last_message.additional_kwargs:
        return "end"
    # Otherwise if there is, we continue
    else:
        return "continue"

# 增加边
workflow.add_conditional_edges(
    "agent", # from节点
    should_continue, # 条件
    {
        # to节点：回到 action
        "continue": "action",
        # to节点：结束
        "end": END
    }
)

### 定义直连边

In [14]:
workflow.add_edge('action', 'agent')

### 设定执行入口

In [15]:
workflow.set_entry_point("agent")

### 编译 graph

In [16]:
app = workflow.compile()

### 使用它！

In [18]:
from langchain_core.messages import HumanMessage

inputs = {"messages": [HumanMessage(content="what is the weather in sf?")]}
app.invoke(inputs)

{'messages': [HumanMessage(content='what is the weather in sf?'),
  AIMessage(content='', additional_kwargs={'function_call': {'arguments': '{\n  "query": "weather in San Francisco"\n}', 'name': 'tavily_search_results_json'}}),
  FunctionMessage(content="[{'url': 'https://www.whereandwhen.net/when/north-america/california/san-francisco-ca/february/', 'content': 'Best time to go to San Francisco? Weather in San Francisco in february 2024  How was the weather last february? Here is the day by day recorded weather in San Francisco in february 2023:  Seasonal average climate and temperature of San Francisco in february  The climate of San Francisco in february is goodSan Francisco at the Beginning of February. Temperatures ranging from 9° C to 14° C translate into very mild mornings and pleasant days. However, chilly people\\xa0...'}]", name='tavily_search_results_json'),
  AIMessage(content="I'm sorry, but I couldn't find the current weather in San Francisco. However, in February, the wea